# Remove iLO  User Account

<div class="alert alert-block alert-info">
<i>Hello User.!<br></i> Welcome to <b>python-iLOrest-library</b> <br> This is the example to <b>a particular user account on HPE iLO system.</b> <br> As you are here let's get it done easily.<br>
    To perform this u need to have <a href = "https://github.com/HewlettPackard/python-ilorest-library" style = "background-color:white">python-ilorest-library</a> installed in your machine.<br>
    And please keep your username and password handy!
    <br>
    <b>Also do execute each successive cell once to avoid any errors!</b>
    
</div>

 <div class = "alert alert-block alert-success">
    <div style = "color:green; font-size:40px;">1.</div>
    <br>
    <div style = "font-size:14px">In the beginning, we are having some import statements that are actually providing us with the required modules and libraries.
        for proper execution of our code.</div>
    </div>

In [ ]:

"""
An example of removing a user account from the HPE iLO systems
"""

import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

#from get_resource_directory import get_resource_directory



<div class = "alert alert-block alert-success">
    <div style = "color:green; font-size:40px;">2.</div>
    <br>
 Below is the remove_ilo_user_account function, after sucessful execution of this function we will be able to delete the information of any user account that is present on the HPE iLO system.
    
<br>
<i>Let me give you a walk through of it!</i>
<br>
    
For removal of any general account from the system we will first have to search for the accounts that we wan't to delete based on any attribute or property that uniquely identifies the account. Upon finding that account we can just make http delete request and we are done!
    
1. In the <b>remove_ilo_user_account</b> function we are first using <b>resource_instances</b> variable to obtain information about the resource directory by using <i>get_resource_directory</i> module.
<br>
    
2. <b>In case the resource directory was disabled</b> we are navigating to the actual location where the all the member account information is stored by using  <a href = "https://restfulapi.net/http-methods/#delete" style="background-color:powderblue;">DELETE</a>request.
 

3. Else if resource directory is enabled then we will simply iterate through all instances inside resources to obtain the <b>account_collection_uri.</b>
    <br>
4. <ul>
    <li>Now we are having the location at which all the member accounts are present. </li><li>Simply we just find out what user account matches the account that has to be deleted based on username of the accounts.</li><li> If we do not find the required account after completion of iteration we print out a message that required acccount was not found on the server side.</li>
    <li>After this we simply make a DELETE request to delete that particular account and save the response obtained in resp variable.</li></ul> If the http status code returned was 200, then we simply print a message that delete operation was success.<br>
    Else if the status code returned was anything else we print out the status code with an error message to get idea about what went wrong.


 </div>

In [7]:
def remove_ilo_user_account(_redfishobj, username_to_delete):

    resource_instances = _redfishobj.get_resource_directory()

    if DISABLE_RESOURCE_DIR or not resource_instances:
        #resource directory is not available so we will navigate through paths manually to obtain
        #account info
        account_service_uri = _redfishobj.root.obj['AccountService']['@odata.id']
        account_service_response = _redfishobj.get(account_service_uri)
        account_collection_uri = account_service_response.obj['Accounts']['@odata.id']
    else:
        #obtain all account instances from resource directory
        for instance in resource_instances:
            if '#ManagerAccountCollection.' in instance['@odata.type']:
                account_collection_uri = instance['@odata.id']
                break

    #find the account to delete
    account_uri_to_delete = None
    account_uris = _redfishobj.get(account_collection_uri)
    for account_uri in account_uris.dict['Members']:
        account = _redfishobj.get(account_uri['@odata.id'])
        if account.dict['UserName'] == username_to_delete:
            account_uri_to_delete = account_uri['@odata.id']
            break

    if not account_uri_to_delete:
        sys.stderr.write("Cannot find account to modify")
        return

    resp = _redfishobj.delete(account_uri_to_delete)

    if resp.status == 400:
        try:
            print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, sort_keys=True))
        except Exception as excp:
            sys.stderr.write("A response error occurred, unable to access iLO Extended Message "\
                             "Info...")
    elif resp.status != 200:
        sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
    else:
        print("Success!\n")
        print(json.dumps(resp.dict, indent=4, sort_keys=True))


<div class="alert alert-block alert-success">
    <div style = "color:green; font-size:40px;">3.</div>
    <br>
<b>The Main function: </b> PLEASE ENTER YOU LOGIN CREDENTIALS HERE TO PROCEED FURTHER<br>
    <ul>
        <li>Once your credentials that are enter everything a redfish obejct is created with the help of RedfishClient class and a session is created and now you will be logged in to the server.</li>
        <li>The <b>ACCOUNT_TO_DELETE</b> contains the username of the account that we need to delete. We are passing the same as an argument while making a call to remove_ilo_user_account function.</li>
        <li>If server is down or Unreachable we get server not reachable message!</li>
        <li>If not then we make a call to <b>remove_ilo_user_account function</b> .</li>
        <li>After getting required information <i><b>Redfish.logout()</b></i>closses the particular session and logs us out of the system.</li>
    </ul>
</div>

In [ ]:
if __name__ == "__main__":
    
    SYSTEM_URL = input("Enter System URL: ")
    LOGIN_ACCOUNT = input("Enter Name: ")
    LOGIN_PASSWORD = input("Enter password: ")
    
    
    # The username of the account to remove
    ACCOUNT_TO_DELETE = "joker"

    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    remove_ilo_user_account(REDFISHOBJ, ACCOUNT_TO_DELETE)

    REDFISHOBJ.logout()


<div class = "alert alert-block alert-warning">
    <b>When running on the server locally use the following commented values</b><br>
 SYSTEM_URL = None<br>
 LOGIN_ACCOUNT = None<br>
 LOGIN_PASSWORD = None<br>

   <b>When running remotely</b> connect using the secured (https://) address,
     account name, and password to send https requests
    <br>
     SYSTEM_URL acceptable examples:<br>
     https://10.0.0.100<br>
     https://ilo.hostname
</div>

<div class = "alert alert-block alert-success">In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library:<a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.Hpe ilorest-api explorer:<a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.Rest api: <a href = "https://restfulapi.net/">LINK</a><br>
    
</div>